In [137]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from keras.models import Sequential
from keras import initializers
from keras.layers import Dropout, Activation, Embedding, Convolution1D, MaxPooling1D, Input, Dense, \
                         BatchNormalization, Flatten, Reshape, Concatenate
from keras.layers.recurrent import LSTM
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras import regularizers
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time 
from sklearn import preprocessing
import os
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn import preprocessing
from collections import deque  
from sklearn.metrics import fbeta_score, make_scorer
import time
import os
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional

np.random.seed(314)


stock = pd.read_csv("AAPL.csv")
news = pd.read_csv("us_equities_news_dataset.csv")


news = news.groupby(['release_date'])['title'].apply(','.join).reset_index()



In [138]:
news = news.groupby(['release_date'])['title'].apply(','.join).reset_index()
news = news[news.release_date.isin(stock.Date)]
stock['Date']= pd.to_datetime(stock['Date']) 
news['release_date']= pd.to_datetime(news['release_date']) 

df_final = pd.merge(left=news, left_on='release_date',
         right=stock, right_on='Date')

df_final

,release_date,title,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-09-21,UPDATE 4 China buys Noble stake seeks commodi...,2009-09-21,6.581786,6.612857,6.486429,6.572143,5.679245,437715600
1,2009-09-22,US STOCKS Wall St to open up as commods rise o...,2009-09-22,6.613929,6.620714,6.530357,6.588572,5.693443,356753600
2,2009-09-23,Nikkei climbs on U S economy optimism after F...,2009-09-23,6.621428,6.746428,6.608214,6.625000,5.724923,593563600
3,2009-09-24,Nikkei gains 1 2 pct after holiday JAL Aiful...,2009-09-24,6.685714,6.703571,6.527500,6.565000,5.673074,550880400
4,2009-09-25,US STOCKS Wall St drops on recovery worry RIM...,2009-09-25,6.500357,6.625000,6.480000,6.513214,5.628324,445239200
5,2009-09-28,GLOBAL MARKETS Yen surge hits Japan shares ri...,2009-09-28,6.566786,6.667143,6.547500,6.648214,5.744982,337444800
6,2009-09-29,Nikkei edges up 0 9 pct but yen worries remai...,2009-09-29,6.668929,6.692857,6.582500,6.620714,5.721219,345385600
7,2009-09-30,CORRECTED CORRECTED CORRECTED GLOBAL MARKETS...,2009-09-30,6.647500,6.658928,6.521786,6.619643,5.720292,539585200
8,2009-10-01,Nikkei hits 2 month low after disappointing US...,2009-10-01,6.619643,6.650714,6.453571,6.459286,5.581721,524711600
9,2009-10-02,European shares hit 4 week low before US jobs ...,2009-10-02,6.478929,6.640714,6.476786,6.603571,5.706404,553308000


In [139]:

stop = stopwords.words('english')


df_final['title'] = df_final['title'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

In [140]:
df_final['release_date'] = pd.to_datetime(df_final['release_date'])
df_final.set_index('release_date', inplace=True)
df_final

,title,Date,Open,High,Low,Close,Adj Close,Volume
release_date,,,,,,,,
2009-09-21,UPDATE 4 China buys Noble stake seeks commodit...,2009-09-21,6.581786,6.612857,6.486429,6.572143,5.679245,437715600
2009-09-22,US STOCKS Wall St open commods rise weak dolla...,2009-09-22,6.613929,6.620714,6.530357,6.588572,5.693443,356753600
2009-09-23,"Nikkei climbs U S economy optimism Fed,Nikkei ...",2009-09-23,6.621428,6.746428,6.608214,6.625000,5.724923,593563600
2009-09-24,"Nikkei gains 1 2 pct holiday JAL Aiful sink,Ni...",2009-09-24,6.685714,6.703571,6.527500,6.565000,5.673074,550880400
2009-09-25,US STOCKS Wall St drops recovery worry RIM res...,2009-09-25,6.500357,6.625000,6.480000,6.513214,5.628324,445239200
2009-09-28,GLOBAL MARKETS Yen surge hits Japan shares ris...,2009-09-28,6.566786,6.667143,6.547500,6.648214,5.744982,337444800
2009-09-29,"Nikkei edges 0 9 pct yen worries remain,Nikkei...",2009-09-29,6.668929,6.692857,6.582500,6.620714,5.721219,345385600
2009-09-30,CORRECTED CORRECTED CORRECTED GLOBAL MARKETS A...,2009-09-30,6.647500,6.658928,6.521786,6.619643,5.720292,539585200
2009-10-01,"Nikkei hits 2 month low disappointing US data,...",2009-10-01,6.619643,6.650714,6.453571,6.459286,5.581721,524711600


In [141]:
df_final[['polarity','subjectivity']] = df_final['title'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))


df_final.drop(['title', 'Date','subjectivity'], axis=1)

,Open,High,Low,Close,Adj Close,Volume,polarity
release_date,,,,,,,
2009-09-21,6.581786,6.612857,6.486429,6.572143,5.679245,437715600,0.056597
2009-09-22,6.613929,6.620714,6.530357,6.588572,5.693443,356753600,0.018947
2009-09-23,6.621428,6.746428,6.608214,6.625000,5.724923,593563600,0.042859
2009-09-24,6.685714,6.703571,6.527500,6.565000,5.673074,550880400,0.075398
2009-09-25,6.500357,6.625000,6.480000,6.513214,5.628324,445239200,0.131481
2009-09-28,6.566786,6.667143,6.547500,6.648214,5.744982,337444800,0.008333
2009-09-29,6.668929,6.692857,6.582500,6.620714,5.721219,345385600,0.027100
2009-09-30,6.647500,6.658928,6.521786,6.619643,5.720292,539585200,0.177381
2009-10-01,6.619643,6.650714,6.453571,6.459286,5.581721,524711600,-0.148611


In [142]:
window = 60
days_after= 1
test_size=0.2
feature_columns=['Open','High','Low','Close','Adj Close','Volume','polarity']


In [143]:
result = {}
df = df_final
result['df'] = df.copy()
scaled = {}
X, y = [], []



for f in feature_columns:
            scaler = preprocessing.MinMaxScaler()
            df[f] = scaler.fit_transform(np.expand_dims(df[f].values, axis=1))
            scaled[f] = scaler

            result["scaled"] = scaled
df['future'] = df['Open'].shift(-days_after)
last_sequence = np.array(df[feature_columns].tail(days_after))
df.dropna(inplace=True)
sequence_data = []
sequences = deque(maxlen=window)



for i, j in zip(df[feature_columns].values, df['future'].values):
    sequences.append(i)
    if len(sequences) == window:
            sequence_data.append([np.array(sequences), j])
last_sequence = list(sequences) + list(last_sequence)
last_sequence = np.array(last_sequence)
result['last_sequence'] = last_sequence




for seq, j in sequence_data:
      X.append(seq)
      y.append(j)
X = np.array(X)
y = np.array(y)
X = X.reshape((X.shape[0], X.shape[2], X.shape[1]))
result["X_train"], result["X_test"], result["y_train"], result["y_test"] = train_test_split(X, y, 
                                                                               test_size=test_size)

In [144]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

monitor = 'val_loss'


callbacks = [
    
#     Stop training when a monitored quantity has stopped improving.
    EarlyStopping(monitor=monitor,
                  patience=10,
                  verbose=1,
                  restore_best_weights=True),

    
    # Reduce learning rate when a metric has stopped improving
    ReduceLROnPlateau(monitor=monitor,
                      factor=0.1,
                      patience=3,
                      verbose=1, min_lr=0)

]

In [145]:
def build_model(sequence_length, units=256, cell=LSTM, n_layers=2, dropout=0.3,
                loss="mean_absolute_error", optimizer="rmsprop", bidirectional=False):
    model = Sequential()
    for i in range(n_layers):
        if i == 0:
            # first layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True), input_shape=(None, sequence_length)))
            else:
                model.add(cell(units, return_sequences=True, input_shape=(None, sequence_length)))
        elif i == n_layers - 1:
            # last layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=False)))
            else:
                model.add(cell(units, return_sequences=False))
        else:
            # hidden layers
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True)))
            else:
                model.add(cell(units, return_sequences=True))
        # add dropout after each layer
        model.add(Dropout(dropout))
    model.add(Dense(1, activation="relu"))
    model.compile(loss=loss, metrics=["mean_absolute_error"], optimizer=optimizer)
    return model

In [146]:
data = result
LOSS = "mean_absolute_error"
OPTIMIZER = "adam"
BIDIRECTIONAL = True

model = build_model(window, loss=LOSS, units=256, cell=LSTM, n_layers=3,
                    dropout=0.4, optimizer=OPTIMIZER, bidirectional=BIDIRECTIONAL)
# some tensorflow callbacks
history = model.fit(data["X_train"], data["y_train"],
                    batch_size=64,
                    epochs=2,
                    validation_data=(data["X_test"], data["y_test"]),
                    callbacks=callbacks,
                    verbose=1)

Epoch 1/2
60/60 [==============================] - 24s 401ms/step - loss: 0.0727 - mean_absolute_error: 0.0727 - val_loss: 0.0263 - val_mean_absolute_error: 0.0263
Epoch 2/2
60/60 [==============================] - 12s 206ms/step - loss: 0.0345 - mean_absolute_error: 0.0345 - val_loss: 0.0222 - val_mean_absolute_error: 0.0222


In [147]:
# evaluate the model
mse, mae = model.evaluate(data["X_test"], data["y_test"], verbose=0)
# calculate the mean absolute error (inverse scaling)
mean_absolute_error = data["scaled"]["Open"].inverse_transform([[mae]])[0][0]
print("Mean Absolute Error:", mean_absolute_error)

Mean Absolute Error: 8.134101095315895


In [148]:
last_sequence = data["last_sequence"][-window:]
column_scaler = data["scaled"]
last_sequence = last_sequence.reshape((last_sequence.shape[1], last_sequence.shape[0]))
last_sequence = np.expand_dims(last_sequence, axis=0)
prediction = model.predict(last_sequence)
predicted_price = column_scaler["Open"].inverse_transform(prediction)[0][0]


In [149]:
predicted_price

76.394295

In [ ]:
monitor = 'val_loss'
batch_size =[16,32,64,128,256]
drop_out = [0.1,0.4,0.8]
optimizer = ['adam','rmsprop']
results_lst =[]
early_stop = EarlyStopping(monitor= monitor,patience=60,verbose=1,restore_best_weights=True)
model_num = 0

results_df = pd.DataFrame(columns=["batch_size", "dropout","rmse","test_loss"])
        
for b in batch_size:
    for d in drop_out:
            model_num += 1
                        
            model_checkpoint = ModelCheckpoint(filepath='t3st_grid'+str(model_num),monitor=monitor,save_best_only=True,save_weights_only=False)

            
            model = build_model(window, loss=LOSS, units=768, cell=LSTM, n_layers=3,
                    dropout=d, optimizer=optimizer, bidirectional=BIDIRECTIONAL)
            

            # evaluate the model
            mse, mae = model.evaluate(data["X_test"], data["y_test"], verbose=0)
            # calculate the mean absolute error (inverse scaling)
            mean_absolute_error = data["scaled"]["Open"].inverse_transform([[mae]])[0][0]
            print("Mean Absolute Error:", mean_absolute_error)


            results_df = results_df.append({"batch_size": b,"dropout": d,"rmse": mean_absolute_error},ignore_index=True)

Mean Absolute Error: 28.867956690399886
Mean Absolute Error: 28.867956690399886
Mean Absolute Error: 28.867956690399886
Mean Absolute Error: 28.830760493545714
Mean Absolute Error: 28.867956690399886


In [153]:
results_df


,batch_size,dropout,rmse,test_loss
0,16.0,0.1,27.587138,NaN
1,16.0,0.4,27.498346,NaN
2,16.0,0.8,28.867957,NaN
3,32.0,0.1,27.993336,NaN
4,32.0,0.4,28.867957,NaN
5,32.0,0.8,25.972385,NaN
6,64.0,0.1,28.144082,NaN
7,64.0,0.4,24.664194,NaN
8,64.0,0.8,27.905377,NaN
9,128.0,0.1,26.865076,NaN


In [ ]:
results_df